In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl
//spark.sparkContext.version

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@5f81e952


Some(http://iuni2:4040)

In [2]:
val WoS = spark.read.format("parquet").load("/WoSraw/WoS2019/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name","static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword",
                      "static_data.fullrecord_metadata.references.reference",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack.grants.grant")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 14 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]

In [12]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct

//WoS2.describe().filter($"summary" === "count").show
val cityJ = WoS2.select("UID","titles","pub_info._pubyear","address_name.address_spec.organizations.organization","address_name.address_spec.city","address_name.address_spec.state","address_name.address_spec.country","address_name.address_spec.full_address")//.filter("city is not null")
//val cityJ2 = cityJ.withColumn("titles", explode(col("title")))
//cityJ2.groupBy("titles._type").count().show

cityJ = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 6 more fields]


lastException: Throwable = null


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 6 more fields]

In [4]:
val InstList = cityJ.withColumn("Institutions", explode(col("organization"))).cache()
InstList.printSchema

root
 |-- UID: string (nullable = true)
 |-- _pubyear: long (nullable = true)
 |-- organization: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _pref: string (nullable = true)
 |-- city: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- country: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- full_address: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Institutions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _pref: string (nullable = true)



InstList = [UID: string, _pubyear: bigint ... 6 more fields]


[UID: string, _pubyear: bigint ... 6 more fields]

In [5]:
val UIDinstEnhanced = InstList.select("UID","_pubyear","full_address","city","state","country","Institutions._VALUE","Institutions._pref").filter(array_contains($"_pref","Y"))
UIDinstEnhanced.describe().filter($"summary" === "count").show

+-------+--------+--------+
|summary|     UID|_pubyear|
+-------+--------+--------+
|  count|94116225|94116225|
+-------+--------+--------+



UIDinstEnhanced = [UID: string, _pubyear: bigint ... 6 more fields]


[UID: string, _pubyear: bigint ... 6 more fields]

In [6]:
InstList.describe().filter($"summary" === "count").show

+-------+---------+---------+
|summary|      UID| _pubyear|
+-------+---------+---------+
|  count|110470160|110470160|
+-------+---------+---------+



In [7]:
UIDinstEnhanced.groupBy("_pubyear").count().orderBy("_pubyear").show(120)

+--------+-------+
|_pubyear|  count|
+--------+-------+
|    1900|    605|
|    1901|    725|
|    1902|    786|
|    1903|    876|
|    1904|    996|
|    1905|   1013|
|    1906|   1111|
|    1907|   1343|
|    1908|   1365|
|    1909|   1416|
|    1910|   1589|
|    1911|   1662|
|    1912|   1708|
|    1913|   1814|
|    1914|   1825|
|    1915|   2032|
|    1916|   2007|
|    1917|   2060|
|    1918|   1665|
|    1919|   1872|
|    1920|   2263|
|    1921|   2945|
|    1922|   3223|
|    1923|   3747|
|    1924|   4106|
|    1925|   4559|
|    1926|   4911|
|    1927|   5472|
|    1928|   5864|
|    1929|   6098|
|    1930|   6957|
|    1931|   6789|
|    1932|   7296|
|    1933|   7778|
|    1934|   8575|
|    1935|   9079|
|    1936|   9701|
|    1937|  10184|
|    1938|  10178|
|    1939|  10813|
|    1940|  10891|
|    1941|  11373|
|    1942|  10108|
|    1943|   8650|
|    1944|   7246|
|    1945|    664|
|    1946|    699|
|    1947|    761|
|    1948|    912|
|    1949|  

In [8]:
UIDinstEnhanced.show(false)

+-------------------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------+-----------+
|UID                |_pubyear|full_address                                                                                                                                                              |city                             |state       |country           |_VALUE                                                                                                                               |_pref      |
+-------------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
val UIDinstEnhanced2 = UIDinstEnhanced.withColumn("full_addresses", array_join($"full_address", "|"))
                                      .withColumn("cities", array_join($"city", "|"))
                                      .withColumn("states", array_join($"state", "|"))
                                      .withColumn("countries", array_join($"country", "|"))
UIDinstEnhanced2.filter($"UID" === "WOS:000257661900001").show(false)

+-------------------+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------+--

UIDinstEnhanced2 = [UID: string, _pubyear: bigint ... 10 more fields]


[UID: string, _pubyear: bigint ... 10 more fields]

In [10]:
val instEnhanced = UIDinstEnhanced2.groupBy("_VALUE").agg(concat_ws(";", collect_list("UID")) as "paperList",
                                                         concat_ws(";", collect_list("full_addresses")) as "full_addresses",
                                                         concat_ws(";", collect_list("cities")) as "cities",
                                                         concat_ws(";", collect_list("states")) as "states",
                                                         concat_ws(";", collect_list("countries")) as "countries")
instEnhanced.describe().filter($"summary" === "count").show
//distribution by paper counts per institution, and fitler the lower ones , rough estimate 20k
//check see if large institiutions over mulitple sites like POLISH ACAD SCI, Polish Academy of Sciences, see how institution enhancement is done

+-------+---------+--------------+------+------+---------+
|summary|paperList|full_addresses|cities|states|countries|
+-------+---------+--------------+------+------+---------+
|  count|   968957|        968957|968957|968957|   968957|
+-------+---------+--------------+------+------+---------+



instEnhanced = [_VALUE: array<string>, paperList: string ... 4 more fields]


[_VALUE: array<string>, paperList: string ... 4 more fields]

In [14]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}

val cityJ3 = cityJ.withColumn("journal", when(col("titles.title").isNotNull, getTitle("source")($"titles.title")).otherwise(lit(null)))
cityJ3.show

+-------------------+--------------------+--------+--------------------+-------------+---------+-------------+--------------------+--------------------+
|                UID|              titles|_pubyear|        organization|         city|    state|      country|        full_address|             journal|
+-------------------+--------------------+--------+--------------------+-------------+---------+-------------+--------------------+--------------------+
|WOS:A1985AUK0600019|[6, [[CLINICA CHI...|    1985|                null|         null|     null|         null|                null|CLINICA CHIMICA ACTA|
|WOS:A1985ATW9400035|[6, [[INSULA-REVI...|    1985|                null|         null|     null|         null|                null|INSULA-REVISTA DE...|
|WOS:A1985AHR7600027|[6, [[ELECTRONICS...|    1985|                null|         null|     null|         null|                null| ELECTRONICS LETTERS|
|WOS:A1985AJM7300014|[5, [[ACTA MORPHO...|    1985|[[[STATE UNIV UTR...|    [UTREC

cityJ3 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]


lastException: Throwable = null
getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 7 more fields]

In [ ]:
val cols = cityJ3.select("UID", "city", "state", "country").columns.filter(_ != "UID")

val cityJ4 = cols.
  foldLeft(cityJ3)( (accDF, c) => 
    accDF.withColumn(c, struct(lit(c).as("name"), col(c).as("value")))
  ).
  select($"UID", array(cols.map(col): _*).as("cityFull"))

cityJ4.show(false)

In [6]:
import org.apache.spark.sql.functions.arrays_zip

val cityJ5 = cityJ3.withColumn(
  "cityFull", arrays_zip($"city", $"state", $"country")
)

cityJ5.show(false)

+-------------------+------------------------------+------------+---------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+------------------------------------------------------------------------------+
|UID                |city                          |state       |country                                |title                                                                                                                                                                                                                                                                                                

cityJ5 = [UID: string, city: array<string> ... 5 more fields]


[UID: string, city: array<string> ... 5 more fields]

In [79]:
import org.apache.spark.sql.functions.{concat, lit}

val cityJ6 = cityJ5.select($"UID",$"journal",explode($"cityFull")).select("UID","journal","col.city","col.state","col.country")
val cityJ7 = cityJ6.withColumn(
  "cityFull", concat($"city", lit(" "), $"state", lit(" "), $"country")
)
//val cityJ8 = cityJ7.groupBy("UID").agg(collect_list("cityFull") as "cityFullList", first("journal") as "journal")
//cityJ8.show(100,false)

cityJ6 = [UID: string, journal: string ... 3 more fields]
cityJ7 = [UID: string, journal: string ... 4 more fields]


[UID: string, journal: string ... 4 more fields]

In [80]:
import org.apache.spark.sql.SaveMode

val cityJ9 = cityJ7.groupBy("cityFull","journal").agg(count(lit(1)).alias("count"))
//cityJ9.show(100,false)
//cityJ9.write.mode(SaveMode.Overwrite).parquet("/user/yan30/WoScityJournal/")

cityJ9 = [cityFull: string, journal: string ... 1 more field]


[cityFull: string, journal: string ... 1 more field]

In [235]:
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.sql.functions.col

val cityJ8 = cityJ6.withColumn(
  "cityFull", regexp_replace(concat($"city", $"state", $"country"),"\\s", "")
)
val cityJ9b = cityJ8.groupBy("cityFull","journal").agg(count(lit(1)).alias("count")).cache()

cityJ8 = [UID: string, journal: string ... 4 more fields]
cityJ9b = [cityFull: string, journal: string ... 1 more field]


[cityFull: string, journal: string ... 1 more field]

In [153]:
//cityJ9b.filter($"cityFull".contains("Nan Kang")).show
//cityJ9b.show(false)
cityJ9b.describe().filter($"summary" === "count").show

+-------+--------+-------+-------+
|summary|cityFull|journal|  count|
+-------+--------+-------+-------+
|  count| 8145390|8312073|8312074|
+-------+--------+-------+-------+



In [8]:
val cityL = spark.read.format("csv").option("header", true).load("/data1/cityList.csv")
val cityList = cityL.withColumn("cityUpper", upper(col("city_rev")));
cityList.show(60)

+------+--------------------+--------------+-------------+-------------+
|  mass|                city|       country|     city_rev|    cityUpper|
+------+--------------------+--------------+-------------+-------------+
|201693|Boston Ma United ...| United States|       Boston|       BOSTON|
|197469|London England Un...|United Kingdom|       London|       LONDON|
|161707|Bethesda Md Unite...| United States|     Bethesda|     BETHESDA|
|141790|   Tokyo Tokyo Japan|         Japan|        Tokyo|        TOKYO|
|137876|    Paris Idf France|        France|        Paris|        PARIS|
|133510|Beijing Beijing C...|         China|      Beijing|      BEIJING|
|105411|Seoul Seoul South...|   South Korea|        Seoul|        SEOUL|
|104059|Los Angeles Ca Un...| United States|  Los Angeles|  LOS ANGELES|
|103052|Chicago Il United...| United States|      Chicago|      CHICAGO|
| 94661|San Francisco Ca ...| United States|San Francisco|SAN FRANCISCO|
| 94371|Moscow Moscow Cit...|        Russia|       

cityL = [mass: string, city: string ... 2 more fields]
cityList = [mass: string, city: string ... 3 more fields]


[mass: string, city: string ... 3 more fields]

In [253]:
val cityM = spark.read.format("csv").option("header", true).load("/data1/cityListMerged.csv")
val cityMerged = cityM.select(" string", " mergeList"," mergeCount").withColumn("mergeLength", length(col(" mergeCount")))
cityMerged.show(60,false)

+-----------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

cityM = [full_address: string,  Latitude: string ... 8 more fields]
cityMerged = [ string: string,  mergeList: string ... 2 more fields]


[ string: string,  mergeList: string ... 2 more fields]

In [254]:
val matchedCityMerge = cityMerged.join(cityList, (cityMerged(" mergeList").contains(cityList("city_rev")) 
                                       && cityMerged(" mergeList").contains(cityList("country"))), "inner")
//matchedCityMerge.groupBy("mass").agg(concat_ws(";", collect_list(" matchList")) as "matchLists").show(60)
//val maxMatch = matchedCityMerge.groupBy("city").agg(max("mergeLength") as "mergeMax")
//val cityMatched = maxMatch.join(cityMerged, cityMerged("city")=== maxMatch("city") &&
                                //cityMerged("mergeLength")=== maxMatch("mergeMax"), "left")
//matchedCityMerge.show(60)
//matchedCityMerge.describe().filter($"summary" === "count").show

matchedCityMerge = [ string: string,  mergeList: string ... 7 more fields]


[ string: string,  mergeList: string ... 7 more fields]

In [260]:
matchedCityMerge.filter($"city".contains("Taipei")).show(false)

+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [255]:
import org.apache.spark.sql.expressions.Window
val w = Window.partitionBy($"city")
val maxMatch2 = matchedCityMerge.withColumn("mergeMax", max("mergeLength").over(w))
    .filter($"mergeMax" === $"mergeLength").orderBy("mergeMax")
    .select(" string", " mergeList", "mass", "city", "country", "city_rev")
    .toDF("string", "matchList", "mass", "city", "country", "city_rev")
maxMatch2.show(60)

+--------------------+--------------------+------+--------------------+--------------+-------------+
|              string|           matchList|  mass|                city|       country|     city_rev|
+--------------------+--------------------+------+--------------------+--------------+-------------+
|Nan Kang District...|"Nan Kang Distric...| 61550|Nan Kang District...|        Taiwan|     Nan Kang|
|Hawthorn Vic Aust...|"Hawthorn Vic Aus...| 66946|Hawthorn Vic Aust...|     Australia|     Hawthorn|
|      Vienna Austria|" Vienna Austria"...| 44553|      Vienna Austria|       Austria|       Vienna|
|Laxenburg Lower A...|"Laxenburg Lower ...| 44773|Laxenburg Lower A...|       Austria|    Laxenburg|
|     Shanghai  China|"Shanghai  China"...| 68939|      Shanghai China|         China|     Shanghai|
|Ankara Ankara Turkey|"Ankara Ankara Tu...| 38768|Ankara Ankara Turkey|        Turkey|       Ankara|
|Beijing Beijing C...|"Beijing Beijing ...|133510|Beijing Beijing C...|         China|     

w = org.apache.spark.sql.expressions.WindowSpec@2ca1c1c1
maxMatch2 = [string: string, matchList: string ... 4 more fields]


[string: string, matchList: string ... 4 more fields]

In [262]:
val matched0 = maxMatch2.join(cityJ9b, maxMatch2("matchList").contains(cityJ9b("cityFull")), "inner")
//matched.show
matched0.describe().filter($"summary" === "count").show

+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+
|summary| string|matchList|   mass|   city|country|city_rev|cityFull|journal|  count|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+
|  count|1678632|  1678632|1678632|1678632|1678632| 1678632| 1678632|1678632|1678632|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+



matched0 = [string: string, matchList: string ... 7 more fields]


lastException: Throwable = null


[string: string, matchList: string ... 7 more fields]

In [245]:
val missing = cityList.select("city").except(matched0.select("city"))
//val missing2 = missing.join(accelList2, missing("Name0")===accelList2("Name0"), "inner").toDF()
missing.show(20,false)

+----+
|city|
+----+
+----+



missing = [city: string]


[city: string]

In [141]:
val journals = spark.read.format("csv").option("header", true).load("/data1/journalName.csv")
val journalWoS = journals.filter($"source_type"==="Thomson")
journalWoS.show(false)

+--------+-------------------------------------------------------------------+-----------+
|journ_id|journal_name                                                       |source_type|
+--------+-------------------------------------------------------------------+-----------+
|188     |4OR-A QUARTERLY JOURNAL OF OPERATIONS RESEARCH                     |Thomson    |
|204     |A + U-ARCHITECTURE AND URBANISM                                    |Thomson    |
|210     |AAA-ARBEITEN AUS ANGLISTIK UND AMERIKANISTIK                       |Thomson    |
|213     |AUGMENTATIVE AND ALTERNATIVE COMMUNICATION                         |Thomson    |
|218     |AAOHN JOURNAL                                                      |Thomson    |
|221     |AAPS JOURNAL                                                       |Thomson    |
|222     |AAPS PHARMSCI                                                      |Thomson    |
|223     |AAPS PHARMSCITECH                                                  |Thomson    |

journals = [journ_id: string, journal_name: string ... 1 more field]
journalWoS = [journ_id: string, journal_name: string ... 1 more field]


[journ_id: string, journal_name: string ... 1 more field]

In [263]:
val matched2 = matched0.join(journalWoS, regexp_replace(matched0("journal"),"\\s", "")
                             ===regexp_replace(journalWoS("journal_name"),"\\s", ""), "left").cache()
//matched.show
matched2.describe().filter($"summary" === "count").show

+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+
|summary| string|matchList|   mass|   city|country|city_rev|cityFull|journal|  count|journ_id|journal_name|source_type|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+
|  count|1678632|  1678632|1678632|1678632|1678632| 1678632| 1678632|1678632|1678632|  826687|      826687|     826687|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+



matched2 = [string: string, matchList: string ... 10 more fields]


[string: string, matchList: string ... 10 more fields]

In [264]:
val JsubMap = spark.read.format("csv").option("header", true).load("/data1/jounral-subdiscipline.csv")
val matched3 = matched2.withColumn("uniqueID",monotonicallyIncreasingId)
                .join(JsubMap, matched2("journ_id")===JsubMap("journ_id"), "left")
matched3.show(false)
//matched3.describe().filter($"summary" === "count").show

+----------------------+-----------------------------------------------------------------------------------------------------------------------------------+-----+----------------------+---------+--------+--------------------+----------------------------------------------------------------------------------------------+-----+--------+--------------------------------------------------------+-----------+--------+--------+------------------------------------------------------------------------+---------+-------+
|string                |matchList                                                                                                                          |mass |city                  |country  |city_rev|cityFull            |journal                                                                                       |count|journ_id|journal_name                                            |source_type|uniqueID|journ_id|formal_name                                                     

JsubMap = [journ_id: string, formal_name: string ... 2 more fields]
matched3 = [string: string, matchList: string ... 15 more fields]


[string: string, matchList: string ... 15 more fields]

In [265]:
val subTable = spark.read.format("csv").option("header", true).load("/data1/subdiscipline.csv")
val matched4 = matched3.join(subTable, matched3("subd_id")===subTable("subd_id"), "left")
//matched3.show(false)
matched4.describe().filter($"summary" === "count").show

+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+--------+--------+-----------+---------+-------+-------+---------+-------+-------+-------+-------+
|summary| string|matchList|   mass|   city|country|city_rev|cityFull|journal|  count|journ_id|journal_name|source_type|uniqueID|journ_id|formal_name|jfraction|subd_id|subd_id|subd_name|disc_id|      x|      y|   size|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+--------+--------+-----------+---------+-------+-------+---------+-------+-------+-------+-------+
|  count|1898945|  1898945|1898945|1898945|1898945| 1898945| 1898945|1898945|1898945| 1047000|     1047000|    1047000| 1898945| 1047000|    1047000|  1047000|1047000|1047000|  1047000|1047000|1047000|1047000|1047000|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+--------+

subTable = [subd_id: string, subd_name: string ... 4 more fields]
matched4 = [string: string, matchList: string ... 21 more fields]


[string: string, matchList: string ... 21 more fields]

In [266]:
val subTable = spark.read.format("csv").option("header", true).load("/data1/discipline.csv")
val matched5 = matched4.join(subTable, matched4("disc_id")===subTable("disc_id"), "left")
//matched3.show(false)
matched5.describe().filter($"summary" === "count").show

+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+--------+--------+-----------+---------+-------+-------+---------+-------+-------+-------+-------+-------+---------+-------+-------+-------+
|summary| string|matchList|   mass|   city|country|city_rev|cityFull|journal|  count|journ_id|journal_name|source_type|uniqueID|journ_id|formal_name|jfraction|subd_id|subd_id|subd_name|disc_id|      x|      y|   size|disc_id|disc_name|      x|      y|  color|
+-------+-------+---------+-------+-------+-------+--------+--------+-------+-------+--------+------------+-----------+--------+--------+-----------+---------+-------+-------+---------+-------+-------+-------+-------+-------+---------+-------+-------+-------+
|  count|1898945|  1898945|1898945|1898945|1898945| 1898945| 1898945|1898945|1898945| 1047000|     1047000|    1047000| 1898945| 1047000|    1047000|  1047000|1047000|1047000|  1047000|1047000|1047000|1047000|1047000|104

subTable = [disc_id: string, disc_name: string ... 3 more fields]
matched5 = [string: string, matchList: string ... 26 more fields]


[string: string, matchList: string ... 26 more fields]

In [183]:
matched5.withColumn("jLength", length(col("jfraction"))).orderBy($"jLength".desc).show(60,false)

+----------------------+--------------------------------------------------------------------------+-----+----------------------+---------+--------+--------------------+------------------------------------------------+-----+--------+------------------------------------------------+-----------+-----------+--------+------------------------------------------------------+------------+-------+-------+--------------------------------+-------+---------+---------+-----+-------+--------------------+---+---+---------+-------+
|string                |matchList                                                                 |mass |city                  |country  |city_rev|cityFull            |journal                                         |count|journ_id|journal_name                                    |source_type|uniqueID   |journ_id|formal_name                                           |jfraction   |subd_id|subd_id|subd_name                       |disc_id|x        |y        |size |disc_id|disc_n

lastException: Throwable = null


In [267]:
val matched5b = matched5.filter($"jfraction".isNotNull).groupBy("uniqueID").pivot("disc_name")
                .agg(sum("jfraction")).na.fill(0)
matched5b.show

+--------------+-------+-------------+--------------+-----------------------------------------+---------+--------------+-----------------------------------------+--------------------+----------+-------------------+--------------+-------------------+---------------+
|      uniqueID|Biology|Biotechnology|Brain Research|Chemical, Mechanical, & Civil Engineering|Chemistry|Earth Sciences|Electrical Engineering & Computer Science|Health Professionals|Humanities|Infectious Diseases|Math & Physics|Medical Specialties|Social Sciences|
+--------------+-------+-------------+--------------+-----------------------------------------+---------+--------------+-----------------------------------------+--------------------+----------+-------------------+--------------+-------------------+---------------+
|49306224558717|    0.0|          0.0|           0.0|                                      0.0|      0.0|           0.0|                                      0.0|                 0.0|       0.0|        

matched5b = [uniqueID: bigint, Biology: double ... 12 more fields]


[uniqueID: bigint, Biology: double ... 12 more fields]

In [268]:
val matched2b = matched2.withColumn("uniqueID",monotonicallyIncreasingId).select("uniqueID","mass","city","country","city_rev","count")
val matched6 = matched2b.join(matched5b, matched2b("uniqueID")===matched5b("uniqueID"), "left").cache()
//matched6.describe().filter($"summary" === "count").show
matched6.show(false)

+-------------+-----+----------------------+---------+--------+-----+-------------+-------+-------------+--------------+-----------------------------------------+---------+--------------+-----------------------------------------+--------------------+----------+-------------------+--------------+-------------------+---------------+
|uniqueID     |mass |city                  |country  |city_rev|count|uniqueID     |Biology|Biotechnology|Brain Research|Chemical, Mechanical, & Civil Engineering|Chemistry|Earth Sciences|Electrical Engineering & Computer Science|Health Professionals|Humanities|Infectious Diseases|Math & Physics|Medical Specialties|Social Sciences|
+-------------+-----+----------------------+---------+--------+-----+-------------+-------+-------------+--------------+-----------------------------------------+---------+--------------+-----------------------------------------+--------------------+----------+-------------------+--------------+-------------------+---------------+
|

matched2b = [uniqueID: bigint, mass: string ... 4 more fields]
matched6 = [uniqueID: bigint, mass: string ... 18 more fields]


[uniqueID: bigint, mass: string ... 18 more fields]

In [284]:
val matched7 = matched6.groupBy("city").agg(sum(matched6("Biology")*matched6("count")) as "Biology"
                                           ,sum(matched6("Biotechnology")*matched6("count")) as "Biotechnology"
                                           ,sum(matched6("Brain Research")*matched6("count")) as "Brain Research"
                                           ,sum(matched6("Chemical, Mechanical, & Civil Engineering")*matched6("count")) as "Chemical, Mechanical, & Civil Engineering"
                                           ,sum(matched6("Chemistry")*matched6("count")) as "Chemistry"
                                           ,sum(matched6("Earth Sciences")*matched6("count")) as "Earth Sciences"
                                           ,sum(matched6("Electrical Engineering & Computer Science")*matched6("count")) as "Electrical Engineering & Computer Science"
                                           ,sum(matched6("Health Professionals")*matched6("count")) as "Health Professionals"
                                           ,sum(matched6("Humanities")*matched6("count")) as "Humanities"
                                           ,sum(matched6("Infectious Diseases")*matched6("count")) as "Infectious Diseases"
                                           ,sum(matched6("Math & Physics")*matched6("count")) as "Math & Physics"
                                           ,sum(matched6("Medical Specialties")*matched6("count")) as "Medical Specialties"
                                           ,sum(matched6("Social Sciences")*matched6("count")) as "Social Sciences"
                                           )
matched7.show(55)

+--------------------+------------------+------------------+------------------+-----------------------------------------+------------------+------------------+-----------------------------------------+--------------------+----------+-------------------+------------------+-------------------+------------------+
|                city|           Biology|     Biotechnology|    Brain Research|Chemical, Mechanical, & Civil Engineering|         Chemistry|    Earth Sciences|Electrical Engineering & Computer Science|Health Professionals|Humanities|Infectious Diseases|    Math & Physics|Medical Specialties|   Social Sciences|
+--------------------+------------------+------------------+------------------+-----------------------------------------+------------------+------------------+-----------------------------------------+--------------------+----------+-------------------+------------------+-------------------+------------------+
|Oxford England Un...|2291.3775989444002|    572.0172708331|   1

matched7 = [city: string, Biology: double ... 12 more fields]


[city: string, Biology: double ... 12 more fields]

In [269]:
//val exprs = matched6.columns.drop(7).map(sum(_)* matched6.columns.tail(4))
//matched6.groupBy("city").agg(exprs.head, exprs.tail: _*).show()
matched6.groupBy("city").sum(matched6.columns.drop(7): _*).show(55)

+--------------------+------------------+------------------+-------------------+----------------------------------------------+------------------+-------------------+----------------------------------------------+-------------------------+---------------+------------------------+-------------------+------------------------+--------------------+
|                city|      sum(Biology)|sum(Biotechnology)|sum(Brain Research)|sum(Chemical, Mechanical, & Civil Engineering)|    sum(Chemistry)|sum(Earth Sciences)|sum(Electrical Engineering & Computer Science)|sum(Health Professionals)|sum(Humanities)|sum(Infectious Diseases)|sum(Math & Physics)|sum(Medical Specialties)|sum(Social Sciences)|
+--------------------+------------------+------------------+-------------------+----------------------------------------------+------------------+-------------------+----------------------------------------------+-------------------------+---------------+------------------------+-------------------+------

In [215]:
matched6.columns.tail(4)

count

In [286]:
matched7.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .save("/tmp/cityProfiles.csv")

lastException: Throwable = null


In [240]:
import sqlContext.implicits._
cityJ9b.filter($"cityFull".contains("Munich")).orderBy($"count".desc).show(60, false)

+---------------------------+------------------------------------------------------------------------------------+-----+
|cityFull                   |journal                                                                             |count|
+---------------------------+------------------------------------------------------------------------------------+-----+
|MunichBavariaGermany       |PLOS ONE                                                                            |36   |
|MunichBavariaGermany       |NEUROBIOLOGY OF AGING                                                               |25   |
|MunichBavariaGermany       |REPRODUCTION FERTILITY AND DEVELOPMENT                                              |20   |
|MunichBavariaGermany       |VALUE IN HEALTH                                                                     |18   |
|MunichBavariaGermany       |PSYCHOSOMATIC MEDICINE                                                              |17   |
|MunichBavariaGermany       |BLO